In [ ]:
import os
import sys
#sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("PostProcess.py"), '..', 'postprocessing')))
#from ..postprocessing.PostProcess import load_process_run3_samples
from HH4b.postprocessing.PostProcess import load_process_run3_samples
#from PostProcess import load_process_run3_samples
from argparse import Namespace 
data_folder = "24Apr23LegacyLowerThresholds_v12_private_signal"
args = Namespace(templates_tag='24May9v2Msd40', data_dir='../../../', tag='24Apr23LegacyLowerThresholds_v12_private_signal', years=['2022EE'], training_years=None, mass='H2PNetMass', bdt_model='24Apr21_legacy_vbf_vars', bdt_config='24Apr21_legacy_vbf_vars', txbb_wps=[0.99, 0.94], bdt_wps=[0.94, 0.68, 0.03], method='sideband', vbf_txbb_wp=0.97, vbf_bdt_wp=0.97, sig_keys=['hh4b', 'vbfhh4b-k2v0'], pt_second=300, bdt_roc=False, control_plots=False, fom_scan=False, fom_scan_bin1=True, fom_scan_bin2=True, fom_scan_vbf=False, templates=False, legacy=True, vbf=False, vbf_priority=False)
bdt_training_keys = ['qcd', 'vbfhh4b-k2v0', 'hh4b', 'ttbar']
ev_dict,_ = load_process_run3_samples(args, year="2022EE", bdt_training_keys=bdt_training_keys, control_plots=False, plot_dir="plot_dir")
print(ev_dict['hh4b'].columns)
"""
python3 PostProcess.py --templates-tag 24May9v2Msd40 \
    --tag 24Apr23LegacyLowerThresholds_v12_private_signal \
    --mass H2PNetMass \
    --legacy --bdt-config 24Apr21_legacy_vbf_vars \
    --bdt-model 24Apr21_legacy_vbf_vars --txbb-wps 0.99 0.94 \
     --bdt-wps 0.94 0.68 0.03 --no-bdt-roc \
     --no-templates --no-fom-scan --no-fom-scan-vbf \
    --data-dir ../../24Apr23LegacyLowerThresholds_v12_private_signal
"""
# make array with event nr, signal category
# make mask with signal category cutoffs, apply to array 

# save as root